基与经验回放池里不同的样本不同的采样权重

In [1]:
import gym
import tensorflow as tf
import numpy as np
import random
from collections import deque

In [2]:
# 衰减因子GAMMA
GAMMA = 0.9

# EPSILON的初始值
INITIAL_EPSILON = 0.5

# EPSILON的最终值
FINAL_EPSILON = 0.01
#经验回放表的大小
REPLAY_SIZE = 10000
# 批量梯度下降的样本数m
BATCH_SIZE = 128

# 更新Q网络的频率
REPLACE_TARGET_FREQ = 10

# 迭代轮次T
EPISODE = 3000
STEP = 300
TEST = 5

In [3]:
class SumTree(object):
    data_pointer = 0
    def __init__(self, capacity):
        self.capacity = capacity  # for all priority values
        self.tree = np.zeros(2 * capacity - 1)
        # [--------------Parent nodes-------------][-------leaves to recode priority-------]
        #             size: capacity - 1                       size: capacity
        self.data = np.zeros(capacity, dtype=object)  # for all transitions
        # [--------------data frame-------------]
        #             size: capacity

    def add(self, p, data):
        tree_idx = self.data_pointer + self.capacity - 1
        self.data[self.data_pointer] = data  # update data_frame
        self.update(tree_idx, p)  # update tree_frame

        self.data_pointer += 1
        if self.data_pointer >= self.capacity:  # replace when exceed the capacity
            self.data_pointer = 0

    def update(self, tree_idx, p):
        change = p - self.tree[tree_idx]
        self.tree[tree_idx] = p
        # then propagate the change through tree
        while tree_idx != 0:    # this method is faster than the recursive loop in the reference code
            tree_idx = (tree_idx - 1) // 2
            self.tree[tree_idx] += change

    def get_leaf(self, v):
        """
        Tree structure and array storage:
        Tree index:
             0         -> storing priority sum
            / \
          1     2
         / \   / \
        3   4 5   6    -> storing priority for transitions
        Array type for storing:
        [0,1,2,3,4,5,6]
        """
        parent_idx = 0
        while True:     # the while loop is faster than the method in the reference code
            cl_idx = 2 * parent_idx + 1         # this leaf's left and right kids
            cr_idx = cl_idx + 1
            if cl_idx >= len(self.tree):        # reach bottom, end search
                leaf_idx = parent_idx
                break
            else:       # downward search, always search for a higher priority node
                if v <= self.tree[cl_idx]:
                    parent_idx = cl_idx
                else:
                    v -= self.tree[cl_idx]
                    parent_idx = cr_idx

        data_idx = leaf_idx - self.capacity + 1
        return leaf_idx, self.tree[leaf_idx], self.data[data_idx]

    @property
    def total_p(self):
        return self.tree[0]  # the root

In [4]:
class Memory(object):
    epsilon = 0.01  # small amount to avoid zero priority
    alpha = 0.6  # [0~1] convert the importance of TD error to priority
    beta = 0.4  # importance-sampling, from initial value increasing to 1
    beta_increment_per_sampling = 0.001
    abs_err_upper = 1.  # clipped abs error

    def __init__(self, capacity):
        self.tree = SumTree(capacity)

    def store(self, transition):
        max_p = np.max(self.tree.tree[-self.tree.capacity:])
        if max_p == 0:
            max_p = self.abs_err_upper
        self.tree.add(max_p, transition)   # set the max p for new p

    def sample(self, n):
        b_idx, b_memory, ISWeights = np.empty((n,), dtype=np.int32), np.empty((n, self.tree.data[0].size)), np.empty((n, 1))
        pri_seg = self.tree.total_p / n       # priority segment
        self.beta = np.min([1., self.beta + self.beta_increment_per_sampling])  # max = 1

        min_prob = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_p     # for later calculate ISweight
        if min_prob == 0:
            min_prob = 0.00001
        for i in range(n):
            a, b = pri_seg * i, pri_seg * (i + 1)
            v = np.random.uniform(a, b)
            idx, p, data = self.tree.get_leaf(v)
            prob = p / self.tree.total_p
            ISWeights[i, 0] = np.power(prob/min_prob, -self.beta)
            b_idx[i], b_memory[i, :] = idx, data
        return b_idx, b_memory, ISWeights

    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.epsilon  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.abs_err_upper)
        ps = np.power(clipped_errors, self.alpha)
        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)

In [5]:
class DQN():
    # DQN Agent
    def __init__(self, env):
        # init experience replay
        self.replay_total = 0
        # init some parameters
        self.time_step = 0
        self.epsilon = INITIAL_EPSILON
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        self.memory = Memory(capacity=REPLAY_SIZE)

        self.create_Q_network()
        self.create_training_method()

        # Init session
        self.session = tf.InteractiveSession()
        self.session.run(tf.global_variables_initializer())

    def create_Q_network(self):
        # input layer
        self.state_input = tf.placeholder("float", [None, self.state_dim])
        self.ISWeights = tf.placeholder(tf.float32, [None, 1])
        # network weights
        with tf.variable_scope('current_net'):
            W1 = self.weight_variable([self.state_dim, 20])
            b1 = self.bias_variable([20])
            W2 = self.weight_variable([20, self.action_dim])
            b2 = self.bias_variable([self.action_dim])

            # hidden layers
            h_layer = tf.nn.relu(tf.matmul(self.state_input, W1) + b1)
            # Q Value layer
            self.Q_value = tf.matmul(h_layer, W2) + b2

        with tf.variable_scope('target_net'):
            W1t = self.weight_variable([self.state_dim, 20])
            b1t = self.bias_variable([20])
            W2t = self.weight_variable([20, self.action_dim])
            b2t = self.bias_variable([self.action_dim])

            # hidden layers
            h_layer_t = tf.nn.relu(tf.matmul(self.state_input, W1t) + b1t)
            # Q Value layer
            self.target_Q_value = tf.matmul(h_layer_t, W2t) + b2t

        t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='current_net')

        with tf.variable_scope('soft_replacement'):
            self.target_replace_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]

    def create_training_method(self):
        self.action_input = tf.placeholder("float", [None, self.action_dim])  # one hot presentation
        self.y_input = tf.placeholder("float", [None])
        Q_action = tf.reduce_sum(tf.multiply(self.Q_value, self.action_input), reduction_indices=1)
        self.cost = tf.reduce_mean(self.ISWeights * (tf.square(self.y_input - Q_action)))
        self.abs_errors = tf.abs(self.y_input - Q_action)
        self.optimizer = tf.train.AdamOptimizer(0.0001).minimize(self.cost)

    def store_transition(self, s, a, r, s_, done):
        transition = np.hstack((s, a, r, s_, done))
        self.memory.store(transition)  # have high priority for newly arrived transition

    def perceive(self, state, action, reward, next_state, done):
        one_hot_action = np.zeros(self.action_dim)
        one_hot_action[action] = 1
        # print(state,one_hot_action,reward,next_state,done)
        self.store_transition(state, one_hot_action, reward, next_state, done)
        self.replay_total += 1
        if self.replay_total > BATCH_SIZE:
            self.train_Q_network()

    def train_Q_network(self):
        self.time_step += 1
        # Step 1: obtain random minibatch from replay memory
        tree_idx, minibatch, ISWeights = self.memory.sample(BATCH_SIZE)
        state_batch = minibatch[:, 0:4]
        action_batch = minibatch[:, 4:6]
        reward_batch = [data[6] for data in minibatch]
        next_state_batch = minibatch[:, 7:11]
        # Step 2: calculate y
        y_batch = []
        current_Q_batch = self.Q_value.eval(feed_dict={self.state_input: next_state_batch})
        max_action_next = np.argmax(current_Q_batch, axis=1)
        target_Q_batch = self.target_Q_value.eval(feed_dict={self.state_input: next_state_batch})

        for i in range(0, BATCH_SIZE):
            done = minibatch[i][11]
            if done:
                y_batch.append(reward_batch[i])
            else:
                target_Q_value = target_Q_batch[i, max_action_next[i]]
                y_batch.append(reward_batch[i] + GAMMA * target_Q_value)

        self.optimizer.run(feed_dict={
            self.y_input: y_batch,
            self.action_input: action_batch,
            self.state_input: state_batch,
            self.ISWeights: ISWeights
        })
        _, abs_errors, _ = self.session.run([self.optimizer, self.abs_errors, self.cost], feed_dict={
            self.y_input: y_batch,
            self.action_input: action_batch,
            self.state_input: state_batch,
            self.ISWeights: ISWeights
        })
        self.memory.batch_update(tree_idx, abs_errors)  # update priority

    def egreedy_action(self, state):
        Q_value = self.Q_value.eval(feed_dict={
            self.state_input: [state]
        })[0]
        if random.random() <= self.epsilon:
            self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 10000
            return random.randint(0, self.action_dim - 1)
        else:
            self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 10000
            return np.argmax(Q_value)

    def action(self, state):
        return np.argmax(self.Q_value.eval(feed_dict={
            self.state_input: [state]
        })[0])

    def update_target_q_network(self, episode):
        # update target Q netowrk
        if episode % REPLACE_TARGET_FREQ == 0:
            self.session.run(self.target_replace_op)
            # print('episode '+str(episode) +', target Q network params replaced!')

    def weight_variable(self, shape):
        initial = tf.truncated_normal(shape)
        return tf.Variable(initial)

    def bias_variable(self, shape):
        initial = tf.constant(0.01, shape=shape)
        return tf.Variable(initial)

In [ ]:
# initialize OpenAI Gym env and dqn agent
env = gym.make('CartPole-v0')
agent = DQN(env)

for episode in range(EPISODE):
    # initialize task
    state = env.reset()
    # Train
    for step in range(STEP):
        action = agent.egreedy_action(state)  # e-greedy action for train
        next_state, reward, done, _ = env.step(action)
        # Define reward for agent
        reward = -1 if done else 0.1
        agent.perceive(state, action, reward, next_state, done)
        state = next_state
        if done:
            break
    # Test every 100 episodes
    if episode % 100 == 0:
        total_reward = 0
        for i in range(TEST):
            state = env.reset()
            for j in range(STEP):
                render = lambda : plt.imshow(env.render(mode='rgb_array'))
                action = agent.action(state)  # direct action for test
                state, reward, done, _ = env.step(action)
                total_reward += reward
                if done:
                    break
        ave_reward = total_reward / TEST
        print ('episode: ', episode, 'Evaluation Average Reward:', ave_reward)
    agent.update_target_q_network(episode)

episode:  0 Evaluation Average Reward: 10.0
episode:  100 Evaluation Average Reward: 31.4
episode:  200 Evaluation Average Reward: 200.0
episode:  300 Evaluation Average Reward: 200.0
episode:  400 Evaluation Average Reward: 198.8
episode:  500 Evaluation Average Reward: 195.6
episode:  600 Evaluation Average Reward: 200.0
episode:  700 Evaluation Average Reward: 200.0
episode:  800 Evaluation Average Reward: 191.6
episode:  900 Evaluation Average Reward: 180.0
episode:  1000 Evaluation Average Reward: 193.4
episode:  1100 Evaluation Average Reward: 165.0
episode:  1200 Evaluation Average Reward: 179.0
episode:  1300 Evaluation Average Reward: 187.2
episode:  1400 Evaluation Average Reward: 191.4
episode:  1500 Evaluation Average Reward: 92.2
episode:  1600 Evaluation Average Reward: 168.8
episode:  1700 Evaluation Average Reward: 163.4
episode:  1800 Evaluation Average Reward: 128.4
episode:  1900 Evaluation Average Reward: 132.4
